In [2]:
test1_df = pd.read_csv('SE_rents2018_test1.csv', index_col=0)
test2_df = pd.read_csv('SE_rents2018_test2.csv', index_col=0)
test3_df = pd.read_csv('SE_rents2018_test3.csv', index_col=0)
train_df = pd.read_csv('SE_rents2018_train.csv', index_col=0)
# bedrooms, bathrooms, size_sqft, and year_built use for linear model.
# borough, neighborhood, has_doorman, and size_sqft for non-linear model. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
url = 'https://api.census.gov/data/2017/acs/acs5?'
params = {'get' : 'NAME,B19013_001E,B25058_001E',
          'for' : 'zip code tabulation area:*',}

r = requests.get(url, params=params)
print(r.url)

https://api.census.gov/data/2017/acs/acs5?get=NAME%2CB19013_001E%2CB25058_001E&for=zip+code+tabulation+area%3A%2A


In [4]:
income_df = pd.DataFrame(r.json()[1:], columns=r.json()[0])
income_df.head(10)

,NAME,B19013_001E,B25058_001E,zip code tabulation area
0,ZCTA5 00601,11757,291,00601
1,ZCTA5 00602,16190,304,00602
2,ZCTA5 00603,16645,306,00603
3,ZCTA5 00606,13387,185,00606
4,ZCTA5 00610,18741,322,00610
5,ZCTA5 00612,17744,310,00612
6,ZCTA5 00616,14918,327,00616
7,ZCTA5 00617,17157,375,00617
8,ZCTA5 00622,16727,392,00622
9,ZCTA5 00623,16401,352,00623


In [5]:

income_df = (income_df
             .rename(
                 columns={
                          'B19013_001E' : 'Median Household Income',
                          'B25058_001E' : 'Median Rent',
                          'zip code tabulation area' : 'addr_zip',
                         }
             )
            )
income_df.head(3)
#income_df.loc['10956']
#income_df.dtypes

,NAME,Median Household Income,Median Rent,addr_zip
0,ZCTA5 00601,11757,291,00601
1,ZCTA5 00602,16190,304,00602
2,ZCTA5 00603,16645,306,00603


In [6]:
income_df['addr_zip'] = income_df['addr_zip'].astype(int)
income_df['Median Rent'] = income_df['addr_zip'].astype(int)
income_df['Median Household Income'] = income_df['Median Household Income'].astype(int)

In [7]:
train_df = train_df.fillna(train_df.median())
income_df = income_df.fillna(income_df.median())
test1_df = test1_df.fillna(test1_df.median())

In [8]:
#TODO I got stuck here
merged_test1_df = test1_df.merge(right=income_df,how='left', on='addr_zip')
merged_train_df = train_df.merge(right=income_df, how='left', on="addr_zip")
# # Here is the link to the data set. https://catalog.data.gov/dataset/demographic-statistics-by-zip-code-acfc9
# demographic_by_zipcode_df = pd.read_csv('Demographic_Statistics_By_Zip_Code.csv', index_col=0)
# print(demographic_by_zipcode_df.columns)
# test_merge_df = pd.merge(test_merge_df,demographic_by_zipcode_df, left_on = test_merge_df.addr_zip , right_index=True, how = 'left')
merged_train_df = merged_train_df.fillna(merged_train_df.median())
merged_test1_df = merged_test1_df.fillna(merged_test1_df.median())

In [9]:
def get_heatmap(df):
    corrmat = df.corr()
    top_corr = corrmat.index
    plt.figure(figsize=(len(df.columns),len(df.columns)))
    df_g = sns.heatmap(df[top_corr].corr(), annot=True, cmap="RdYlGn")
    return df_g

In [10]:
# import numpy

# #using a heatmap to select meaningful features
# #columns = test1_df[:,0:29]
# #target = test1_df.rent
# # get_heatmap(test1_df)


# columns_needed = demographic_by_zipcode_df.columns
# columns_needed = numpy.append (columns_needed, ['rent'])

# print(columns_needed)
# test_heatmap_df = merged_train_df.filter(columns_needed, axis=1)

# get_heatmap(test_heatmap_df)

In [11]:
# sns.pairplot(test_merge_df, size=2.5)
# plt.tight_layout()
# plt.show()

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def train_linear_reg_model(X_train, y_train, X_test, y_test):
    """Trains and test linear regression model and returns mean_squared_error of
    y_test and y_predicted. 
    ----------
    X_train : data frame, shape = [n_samples, n_features]
        n_samples the number of samples
        n_features the number of features for each sample.
        
    y_train : data frame, shape = [n_samples]
        n_sample target values.
        
    X_test : data frame, shape = [n_samples, n_features]
        n_samples the number of samples
        n_features the number of features for each sample.
        
    y_test : data frame, shape = [n_samples]
        n_sample target values.
        
    Return 
        (data frame) y_prediction: predictions made from X_test
        (float) mean_squared_error of (y_test and y_prediction)
        (LinearRegression) returns the linear regression model.
    """

    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_prediction = lr.predict(X_test)
    
    return lr ,y_prediction, mean_squared_error(y_test, y_prediction)

In [13]:
columns_needed = ['bathrooms', 'size_sqft', 'bin','bbl','has_doorman','floor_count','has_gym','rent']
features = ['bathrooms', 'size_sqft','bin','bbl','has_doorman','floor_count','has_gym']

linear_reg_train_df = train_df[columns_needed]

# Dataset contains one null value in bin. So I decided to drop it.
linear_reg_train_df = linear_reg_train_df.dropna(axis=0)

# Getting the features for training.
train_dropped_na = linear_reg_train_df[features]
# Getting the target columns
train_dropped_na_target = linear_reg_train_df[['rent']]

# Extracting the columns needed for testing. Useing test1.csv file.
test1_lr_df = test1_df[features]
# Getting the target columns to get accuracy of predictions. 
test1_lr_target = test1_df[['rent']]

lr, y_prediction, mse = train_linear_reg_model(train_dropped_na, train_dropped_na_target, test1_lr_df, test1_lr_target)
print("Mean square error of test1: " + str(mse))

Mean square error of test1: 3353369.3244725037


In [14]:
test2_lr_df = test2_df[features]
test2_rent_predictions = lr.predict(test2_lr_df)
test2_rent_predictions = pd.DataFrame(test2_rent_predictions, index=test2_lr_df.index, columns=['rent'])
test2_rent_predictions
# send to a csv file
test2_rent_predictions.to_csv('rent_predictions.csv')


In [15]:
demographic_by_zipcode_df = pd.read_csv('Demographic_Statistics_By_Zip_Code.csv', index_col=0)
print(demographic_by_zipcode_df.columns)
merged_df = pd.merge(test1_df,demographic_by_zipcode_df, left_on = test1_df.addr_zip , right_index=True, how = 'left')

Index(['COUNT PARTICIPANTS', 'COUNT FEMALE', 'PERCENT FEMALE', 'COUNT MALE',
       'PERCENT MALE', 'COUNT GENDER UNKNOWN', 'PERCENT GENDER UNKNOWN',
       'COUNT GENDER TOTAL', 'PERCENT GENDER TOTAL', 'COUNT PACIFIC ISLANDER',
       'PERCENT PACIFIC ISLANDER', 'COUNT HISPANIC LATINO',
       'PERCENT HISPANIC LATINO', 'COUNT AMERICAN INDIAN',
       'PERCENT AMERICAN INDIAN', 'COUNT ASIAN NON HISPANIC',
       'PERCENT ASIAN NON HISPANIC', 'COUNT WHITE NON HISPANIC',
       'PERCENT WHITE NON HISPANIC', 'COUNT BLACK NON HISPANIC',
       'PERCENT BLACK NON HISPANIC', 'COUNT OTHER ETHNICITY',
       'PERCENT OTHER ETHNICITY', 'COUNT ETHNICITY UNKNOWN',
       'PERCENT ETHNICITY UNKNOWN', 'COUNT ETHNICITY TOTAL',
       'PERCENT ETHNICITY TOTAL', 'COUNT PERMANENT RESIDENT ALIEN',
       'PERCENT PERMANENT RESIDENT ALIEN', 'COUNT US CITIZEN',
       'PERCENT US CITIZEN', 'COUNT OTHER CITIZEN STATUS',
       'PERCENT OTHER CITIZEN STATUS', 'COUNT CITIZEN STATUS UNKNOWN',
       'PERCENT

In [16]:
features = ['bathrooms', 'size_sqft','bin','bbl','has_doorman','floor_count','has_gym', 'rent', 'Median Household Income']
features_df = merged_train_df[features];
features_df.head()

,bathrooms,size_sqft,bin,bbl,has_doorman,floor_count,has_gym,rent,Median Household Income
0,3.0,1300.0,3117619.0,3051240001,0,6.0,0,3925,48495.0
1,1.0,475.0,1001883.0,1001737504,1,17.0,1,3350,106056.0
2,3.0,1886.0,1039676.0,1013440001,1,33.0,1,11500,133208.0
3,1.0,827.0,1015919.0,1008380001,1,24.0,1,3772,107861.0
4,1.0,1200.0,3053335.0,3018510053,0,4.0,0,2595,56025.0


In [17]:
#hot encode classification variables
features_df = pd.get_dummies(features_df)
features_df.head(20)

,bathrooms,size_sqft,bin,bbl,has_doorman,floor_count,has_gym,rent,Median Household Income
0,3.0,1300.0,3117619.0,3051240001,0,6.0,0,3925,48495.0
1,1.0,475.0,1001883.0,1001737504,1,17.0,1,3350,106056.0
2,3.0,1886.0,1039676.0,1013440001,1,33.0,1,11500,133208.0
3,1.0,827.0,1015919.0,1008380001,1,24.0,1,3772,107861.0
4,1.0,1200.0,3053335.0,3018510053,0,4.0,0,2595,56025.0
5,1.0,0.0,1055734.0,1018440060,0,6.0,0,2695,82352.0
6,1.0,850.0,4035893.0,4014610057,0,3.0,0,2600,53963.0
7,1.0,500.0,4000000.0,4005060029,1,0.0,1,2075,60436.0
8,1.0,960.0,1026932.0,1010720050,1,14.0,1,4996,93707.0
9,1.0,400.0,1038562.0,1013277502,1,43.0,1,2800,133208.0


In [18]:
# the feature we are trying to predict
labels = np.array(features_df['rent'])
features_df = features_df.drop('rent', axis=1)
# convertint to list
feature_list = list(features_df.columns)
features_df = np.array(features_df)

In [19]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(features_df, merged_train_df.rent);

In [20]:
columns_needed = ['bathrooms', 'size_sqft','bin','bbl','has_doorman','floor_count','has_gym', 'Median Household Income']

In [24]:
df = merged_test1_df[columns_needed].values
y_val = merged_test1_df['rent']
predictions = rf.predict(df)

In [25]:
mean_squared_error(y_val, predictions)

1797974.4239098649